# **Itembased collaborative filtering**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ratings=pd.read_csv('/content/drive/My Drive/Dataset/Ratings .csv')
movies=pd.read_csv('/content/drive/My Drive/Dataset/movies.csv')
userdata=pd.read_csv('/content/drive/My Drive/Dataset/user_data.csv')

In [ ]:
movies=pd.read_csv('/content/drive/My Drive/Dataset/movies.csv')
movies

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
ratings=pd.read_csv('/content/drive/My Drive/Dataset/Ratings .csv')
ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
userdata=pd.read_csv('/content/drive/My Drive/Dataset/user_data.csv')
userdata

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,1060


In [ ]:
df=ratings.merge(userdata, on='UserID',how='left').merge(movies, on='MovieID',how='left')
df

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,956715648,M,25,6,11106,Sophie's Choice (1982),Drama


In [ ]:
df=df.drop(columns='Timestamp')
df

,UserID,MovieID,Rating,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,M,25,6,11106,Sophie's Choice (1982),Drama


In [ ]:
df.shape

(1000209, 9)

##***KNNBasic***

In [ ]:
!pip install surprise

     |████████████████████████████████| 6.5MB 28kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675743 sha256=06a04343c2ddc3c80985e442aa83e43bf2f2adfe1b859cbfc2dd60a12a32acf5
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader , KNNBasic
from collections import defaultdict

In [ ]:
reader = Reader()
#Loads from Dataframe into data
data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

In [ ]:
#Compute similarity between items
methods = {
    'name':'cosine',
    'user_based': False
}
algo = KNNBasic(sim_options=methods)

In [ ]:
trainset = data.build_full_trainset()
#Fit KNNBasic
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [ ]:
type(predictions)


list

In [ ]:
predictions[1]

Prediction(uid=1, iid=3068, r_ui=3.581564453029317, est=4.2503606620636765, details={'actual_k': 40, 'was_impossible': False})

In [ ]:
len(predictions)

21384031

In [ ]:
def get_top_n(predictions, n=10):
# First map the predictions to each user.
    top_n = defaultdict(list)
    for UserID, MovieID, true_r, est, _ in predictions:
        top_n[UserID].append((MovieID, est))
        # Then sort the predictions for each user and retrieve the k highest ones.
    for UserID, ratings in top_n.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[UserID] = ratings[:n]

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)
# Print the recommended items for each user
for UserID, ratings in top_n.items():
    print(UserID, [MovieID for (MovieID, _) in ratings])

Streaming output truncated to the last 5000 lines.
1042 [826, 3172, 989, 3890, 642, 3290, 1709, 3297, 3233, 1579]
1043 [3280, 1915, 3172, 2619, 1360, 439, 3607, 3772, 734, 3493]
1044 [655, 3842, 1830, 3656, 924, 3530, 296, 471, 3172, 1245]
1045 [1579, 3209, 3779, 1832, 730, 758, 1724, 2619, 3277, 672]
1046 [3280, 3647, 1830, 3172, 1062, 2619, 3530, 545, 2909, 1039]
1047 [133, 1832, 402, 2308, 3656, 3530, 2591, 872, 3842, 624]
1048 [402, 3647, 3890, 624, 545, 2308, 2438, 3323, 127, 3382]
1049 [3209, 2591, 3890, 1316, 1832, 3779, 3323, 127, 3382, 3290]
1050 [402, 545, 3172, 624, 1200, 2791, 1221, 1214, 1090, 1197]
1051 [872, 3172, 624, 1214, 1374, 2591, 557, 3290, 1709, 17]
1052 [1579, 1830, 872, 3172, 133, 3290, 1709, 1832, 3607, 3458]
1053 [3172, 872, 912, 1358, 624, 50, 1278, 1208, 1945, 545]
1054 [3280, 1039, 1579, 545, 3647, 3297, 1915, 3779, 791, 3172]
1055 [655, 1830, 3164, 989, 3890, 642, 3607, 889, 545, 1579]
1056 [989, 3656, 826, 889, 655, 1062, 226, 3277, 3607, 157]
1057 [1832

In [ ]:
#6040 [926, 3468, 3462, 1939, 3546, 1260, 1256, 3266, 971, 951]
movies.loc[movies['MovieID'].isin(['926','3468','3462','1939','3546','1260','1256','3266','971','951'])]

,MovieID,Title,Genres
914,926,All About Eve (1950),Drama
939,951,His Girl Friday (1940),Comedy
959,971,Cat on a Hot Tin Roof (1958),Drama
1236,1256,Duck Soup (1933),Comedy|War
1240,1260,M (1931),Crime|Film-Noir|Thriller
1870,1939,"Best Years of Our Lives, The (1946)",Drama|War
3197,3266,Man Bites Dog (C'est arriv� pr�s de chez vous)...,Action|Comedy|Crime|Drama
3393,3462,Modern Times (1936),Comedy
3399,3468,"Hustler, The (1961)",Drama
3477,3546,What Ever Happened to Baby Jane? (1962),Drama|Thriller


In [ ]:
from surprise import accuracy
accuracy.rmse(predictions, verbose=True)

RMSE: 0.5117


0.5116950053505896

In [ ]:
accuracy.mae(predictions, verbose=True)

MAE:  0.4127


0.4127177956690517

##***KNNWithMeans***

In [ ]:
!pip install surprise

In [ ]:
from surprise import Dataset, Reader , KNNWithMeans
from collections import defaultdict

In [ ]:
reader = Reader()
#Loads from Dataframe into data
data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

In [ ]:
#Compute similarity between items
methods = {
    'name':'cosine',
    'user_based': False
}
algo = KNNWithMeans(sim_options=methods)

In [ ]:
trainset = data.build_full_trainset()
#Fit KNNWithMeans
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [ ]:
def get_top_n(predictions, n=10):
# First map the predictions to each user.
    top_n = defaultdict(list)
    for UserID, MovieID, true_r, est, _ in predictions:
        top_n[UserID].append((MovieID, est))
        # Then sort the predictions for each user and retrieve the k highest ones.
    for UserID, ratings in top_n.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[UserID] = ratings[:n]

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)
# Print the recommended items for each user
for UserID, ratings in top_n.items():
    print(UserID, [MovieID for (MovieID, _) in ratings])

Streaming output truncated to the last 5000 lines.
1042 [989, 3233, 3172, 3382, 3607, 2480, 787, 3881, 3656, 3245]
1043 [3280, 1915, 1830, 3172, 3656, 3607, 787, 989, 3245, 3382]
1044 [3280, 1830, 3172, 3656, 3233, 787, 989, 3881, 3245, 3607]
1045 [758, 989, 1830, 3233, 3172, 3656, 3382, 3607, 53, 787]
1046 [3280, 787, 3245, 3647, 1830, 3172, 3656, 3382, 3607, 3233]
1047 [3280, 787, 3245, 3881, 3233, 3656, 3382, 3607, 2503, 53]
1048 [3280, 989, 1830, 3233, 3656, 3382, 787, 3607, 3881, 3647]
1049 [3233, 3656, 3382, 989, 1316, 3881, 787, 2480, 53, 3245]
1050 [3881, 3172, 1830, 3233, 3607, 3280, 787, 3888, 3245, 989]
1051 [3280, 787, 3245, 557, 989, 3881, 3233, 3172, 3656, 3607]
1052 [3280, 787, 3245, 989, 1830, 3881, 3233, 3172, 3656, 3607]
1053 [787, 989, 3233, 3172, 3656, 3607, 1830, 3881, 53, 3888]
1054 [1193, 1197, 2804, 919, 720, 527, 1545, 2762, 2692, 260]
1055 [3280, 989, 1830, 3881, 3607, 3233, 787, 3382, 3172, 3164]
1056 [3280, 668, 1117, 989, 3656, 3382, 3607, 1830, 3245, 787]


In [ ]:
a=int(input("Enter UserID : "))
top_n[a]

Enter UserID : 6040


[(3656, 5),
 (3382, 5),
 (1830, 4.993292062128522),
 (3172, 4.849871932256497),
 (787, 4.786235565494341),
 (670, 4.765567918714496),
 (1178, 4.764119087901333),
 (2905, 4.763343131883925),
 (989, 4.6934857360802695),
 (926, 4.688455974718396)]

In [ ]:
movies.loc[movies['MovieID'].isin(['3656','3382','1830','3172','787','670','1178','2905','989','926'])]

,MovieID,Title,Genres
664,670,"World of Apu, The (Apur Sansar) (1959)",Drama
777,787,"Gate of Heavenly Peace, The (1995)",Documentary
914,926,All About Eve (1950),Drama
977,989,Schlafes Bruder (Brother of Sleep) (1995),Drama
1162,1178,Paths of Glory (1957),Drama|War
1762,1830,Follow the Bitch (1998),Comedy
2836,2905,Sanjuro (1962),Action|Adventure
3103,3172,Ulysses (Ulisse) (1954),Adventure
3313,3382,Song of Freedom (1936),Drama
3587,3656,Lured (1947),Crime


In [ ]:
from surprise import accuracy
accuracy.rmse(predictions, verbose=True)

RMSE: 0.8325


0.8325201050066152

In [ ]:
accuracy.mae(predictions, verbose=True)

MAE:  0.6441


0.6440720669637623

##***KNNBaseline***

In [ ]:
!pip install surprise

In [ ]:
from surprise import Dataset, Reader , KNNBaseline
from collections import defaultdict

In [ ]:
reader = Reader()
#Loads from Dataframe into data
data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

In [ ]:
#Compute similarity between items
methods = {
    'name':'cosine',
    'user_based': False
}
algo = KNNBaseline(sim_options=methods)

In [ ]:
trainset = data.build_full_trainset()
#Fit KNNBaseline
algo.fit(trainset)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [ ]:
def get_top_n(predictions, n=10):
# First map the predictions to each user.
    top_n = defaultdict(list)
    for UserID, MovieID, true_r, est, _ in predictions:
        top_n[UserID].append((MovieID, est))
        # Then sort the predictions for each user and retrieve the k highest ones.
    for UserID, ratings in top_n.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[UserID] = ratings[:n]

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)
# Print the recommended items for each user
for UserID, ratings in top_n.items():
    print(UserID, [MovieID for (MovieID, _) in ratings])

Streaming output truncated to the last 5000 lines.
1042 [989, 642, 3172, 2905, 2019, 922, 745, 1148, 826, 3089]
1043 [1915, 3280, 2019, 318, 2905, 922, 858, 745, 1148, 527]
1044 [2019, 1212, 3030, 750, 50, 745, 922, 720, 3134, 1252]
1045 [3779, 758, 2905, 2019, 318, 3209, 50, 858, 922, 745]
1046 [3280, 3647, 1830, 2019, 318, 2905, 922, 50, 858, 745]
1047 [2019, 2905, 3022, 2731, 50, 318, 1148, 1212, 745, 858]
1048 [2019, 2905, 922, 858, 745, 50, 1148, 904, 1178, 1212]
1049 [1316, 2019, 318, 2905, 922, 50, 858, 745, 1148, 904]
1050 [858, 1198, 2019, 260, 1221, 904, 3435, 1207, 922, 908]
1051 [3435, 899, 720, 905, 922, 1214, 1260, 1361, 2019, 923]
1052 [318, 745, 2905, 260, 1262, 527, 2019, 858, 904, 3030]
1053 [318, 50, 912, 1262, 1207, 750, 1234, 1204, 1945, 527]
1054 [1193, 1197, 2804, 919, 720, 527, 2762, 2692, 260, 1207]
1055 [1830, 655, 2019, 922, 720, 2905, 1178, 904, 745, 912]
1056 [668, 2905, 2732, 1117, 2609, 3134, 326, 1949, 2019, 989]
1057 [2905, 1148, 1832, 3089, 922, 745, 7

In [ ]:
a=int(input("Enter UserID : "))
top_n[a]

Enter UserID : 6040


[(1178, 4.767027743544161),
 (926, 4.7334472521497695),
 (922, 4.68837358056517),
 (1260, 4.680735414707139),
 (2905, 4.662253005501523),
 (1207, 4.661369665136493),
 (898, 4.656120918543458),
 (3030, 4.651537022699175),
 (2731, 4.643051059190308),
 (3022, 4.622221784691693)]

In [ ]:
movies.loc[movies['MovieID'].isin(['1178','926','922','1260','2905','1207','898','3030','2731','3022'])]

,MovieID,Title,Genres
886,898,"Philadelphia Story, The (1940)",Comedy|Romance
910,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir
914,926,All About Eve (1950),Drama
1162,1178,Paths of Glory (1957),Drama|War
1189,1207,To Kill a Mockingbird (1962),Drama
1240,1260,M (1931),Crime|Film-Noir|Thriller
2662,2731,"400 Blows, The (Les Quatre cents coups) (1959)",Drama
2836,2905,Sanjuro (1962),Action|Adventure
2953,3022,"General, The (1927)",Comedy
2961,3030,Yojimbo (1961),Comedy|Drama|Western


In [ ]:
from surprise import accuracy
accuracy.rmse(predictions, verbose=True)

RMSE: 0.6737


0.6737178769207075

In [ ]:
accuracy.mae(predictions, verbose=True)

MAE:  0.5253


0.5252953355724894

##***KNNBaseline using pearson_baseline***

In [ ]:
!pip install surprise

In [ ]:
from surprise import Dataset, Reader , KNNBaseline
from collections import defaultdict

In [ ]:
reader = Reader()
#Loads from Dataframe into data
data = Dataset.load_from_df(df[['UserID', 'MovieID', 'Rating']], reader)

In [ ]:
#Compute similarity between items
methods = {
    'name':'pearson_baseline',
    'user_based': False
}
algo = KNNBaseline(sim_options=methods)

In [ ]:
trainset = data.build_full_trainset()
#Fit KNNBaseline
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [ ]:
def get_top_n(predictions, n=10):
# First map the predictions to each user.
    top_n = defaultdict(list)
    for UserID, MovieID, true_r, est, _ in predictions:
        top_n[UserID].append((MovieID, est))
        # Then sort the predictions for each user and retrieve the k highest ones.
    for UserID, ratings in top_n.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[UserID] = ratings[:n]

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)
# Print the recommended items for each user
for UserID, ratings in top_n.items():
    print(UserID, [MovieID for (MovieID, _) in ratings])

Streaming output truncated to the last 5000 lines.
1042 [53, 787, 3469, 3353, 1198, 1236, 2329, 1704, 1923, 3147]
1043 [3216, 3828, 1145, 3687, 623, 2895, 3493, 734, 2731, 2203]
1044 [1284, 2019, 1212, 3022, 904, 3435, 50, 1434, 930, 750]
1045 [1704, 572, 1234, 110, 1861, 3486, 3147, 1304, 2343, 953]
1046 [578, 672, 789, 1851, 3084, 50, 884, 922, 318, 678]
1047 [1217, 1212, 2186, 2905, 1248, 527, 1193, 50, 2925, 318]
1048 [904, 1178, 2019, 912, 1262, 858, 750, 922, 745, 557]
1049 [3307, 3022, 130, 3429, 1247, 1207, 951, 2019, 720, 750]
1050 [557, 260, 1198, 2343, 3429, 1787, 1136, 3030, 2905, 858]
1051 [3435, 1178, 2905, 1361, 720, 2208, 3022, 3134, 899, 2360]
1052 [1198, 2019, 953, 745, 2357, 904, 557, 1234, 356, 527]
1053 [912, 3030, 1276, 3493, 3233, 2905, 527, 1234, 593, 953]
1054 [3408, 1287, 2918, 2791, 527, 1545, 2762, 260, 1207, 2028]
1055 [3353, 134, 745, 2905, 53, 720, 1198, 246, 296, 318]
1056 [1248, 922, 1949, 3201, 668, 623, 2731, 58, 666, 2895]
1057 [1148, 557, 922, 2731,

In [ ]:
a=int(input("Enter UserID : "))
top_n[a]

Enter UserID : 6040


[(1260, 4.709112481372669),
 (1178, 4.669237315605427),
 (3307, 4.58314917325879),
 (2731, 4.5788136963823405),
 (3629, 4.534039393739649),
 (3022, 4.531103273940926),
 (1267, 4.528414321140268),
 (3030, 4.5222891722577465),
 (3730, 4.5056842367160455),
 (905, 4.5024242377359736)]

In [ ]:
movies.loc[movies['MovieID'].isin(['1260','1178','3307','2731','3629','3022','1267','3030','3730','905'])]

,MovieID,Title,Genres
893,905,It Happened One Night (1934),Comedy
1162,1178,Paths of Glory (1957),Drama|War
1240,1260,M (1931),Crime|Film-Noir|Thriller
1247,1267,"Manchurian Candidate, The (1962)",Film-Noir|Thriller
2662,2731,"400 Blows, The (Les Quatre cents coups) (1959)",Drama
2953,3022,"General, The (1927)",Comedy
2961,3030,Yojimbo (1961),Comedy|Drama|Western
3238,3307,City Lights (1931),Comedy|Drama|Romance
3560,3629,"Gold Rush, The (1925)",Comedy
3661,3730,"Conversation, The (1974)",Drama|Mystery


In [ ]:
from surprise import accuracy
accuracy.rmse(predictions, verbose=True)

RMSE: 0.7102


0.7102025065791301

In [ ]:
accuracy.mae(predictions, verbose=True)

MAE:  0.5505


0.5504937858551966

# Results

*KNNBasic*

*   RMSE: 0.5117
*   MAE:  0.4127



*KNNWithMeans*

*   RMSE: 0.8325
*   MAE:  0.6441


*KNNBaseline*

*   RMSE: 0.6737
*   MAE:  0.5253

*KNNBaseline* using pearson_baseline


*   RMSE: 0.7102
*   MAE:  0.5505



From these algorithm the best prediction gives *KNNBasic* with RMSE: 0.5117



